<a href="https://colab.research.google.com/github/shyDaniel/Binary_Classification/blob/master/Generative_probabilistic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **GENERATIVE PROBABILISTIC MODEL**

Kaggle: https://www.kaggle.com/c/ml2020spring-hw2/overview

Hanyu Song 03/24/2020

In [4]:
import pandas as pd
import numpy as np 

!gdown --id '1JuYMZyt0-jwVIwiVs-mgQE8i_O3QUyT3' --output Xtrain.zip
!gdown --id '1iVZaPjgTmwwp683hV8cw-7Q2r3AunasO' --output Xtest.zip
!gdown --id '1C0eGiL5azG49lnS45LxCdbrY0Ce4nNU2'
!unzip Xtrain.zip
!unzip Xtest.zip

Downloading...
From: https://drive.google.com/uc?id=1JuYMZyt0-jwVIwiVs-mgQE8i_O3QUyT3
To: /content/Xtrain.zip
2.79MB [00:00, 83.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1iVZaPjgTmwwp683hV8cw-7Q2r3AunasO
To: /content/Xtest.zip
100% 1.42M/1.42M [00:00<00:00, 90.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1C0eGiL5azG49lnS45LxCdbrY0Ce4nNU2
To: /content/Y_train
100% 423k/423k [00:00<00:00, 56.8MB/s]
Archive:  Xtrain.zip
  inflating: X_train                 
Archive:  Xtest.zip
  inflating: X_test                  


In [0]:
X_train_fpath = '/content/X_train'
Y_train_fpath = '/content/Y_train'
X_test_fpath = '/content/X_test'

with open(X_train_fpath) as f:
    next(f)
    X_train = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)
with open(Y_train_fpath) as f:
    next(f)
    Y_train = np.array([line.strip('\n').split(',')[1] for line in f], dtype = float)
with open(X_test_fpath) as f:
    next(f)
    X_test = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)

**NORMALIZATION**

In [0]:
def _normalize(X, train = True, specified_column = None, X_mean = None, X_std = None):
    # This function normalizes specific columns of X.
    # The mean and standard variance of training data will be reused when processing testing data.
    #
    # Arguments:
    #     X: data to be processed
    #     train: 'True' when processing training data, 'False' for testing data
    #     specific_column: indexes of the columns that will be normalized. If 'None', all columns
    #         will be normalized.
    #     X_mean: mean value of training data, used when train = 'False'
    #     X_std: standard deviation of training data, used when train = 'False'
    # Outputs:
    #     X: normalized data
    #     X_mean: computed mean value of training data
    #     X_std: computed standard deviation of training data

    if specified_column == None:
        specified_column = np.arange(X.shape[1])
    if train:
        X_mean = np.mean(X[:, specified_column] ,0).reshape(1, -1)
        X_std  = np.std(X[:, specified_column], 0).reshape(1, -1)

    X[:,specified_column] = (X[:, specified_column] - X_mean) / (X_std + 1e-8)
     
    return X, X_mean, X_std

def _predict(X, w, b):
    # This function returns a truth value prediction for each row of X 
    # by rounding the result of logistic regression function.
    return np.round(_f(X, w, b)).astype(np.int)

def _f(X, w, b):
    # This is the logistic regression function, parameterized by w and b
    #
    # Arguements:
    #     X: input data, shape = [batch_size, data_dimension]
    #     w: weight vector, shape = [data_dimension, ]
    #     b: bias, scalar
    # Output:
    #     predicted probability of each row of X being positively labeled, shape = [batch_size, ]
    return _sigmoid(np.matmul(X, w) + b)

def _sigmoid(z):
    # Sigmoid function can be used to calculate probability.
    # To avoid overflow, minimum/maximum output value is set.
    return np.clip(1 / (1.0 + np.exp(-z)), 1e-8, 1 - (1e-8))

def _accuracy(Y_pred, Y_label):
    # This function calculates prediction accuracy
    acc = 1 - np.mean(np.abs(Y_pred - Y_label))
    return acc

In [0]:
# Normalize training and testing data
X_train, X_mean, X_std = _normalize(X_train, train = True)
X_test, _, _= _normalize(X_test, train = False, specified_column = None, X_mean = X_mean, X_std = X_std)

**Mean and Covariance**

In [0]:
# Compute in-class mean
X_train_0 = np.array([x for x, y in zip(X_train, Y_train) if y == 0])
X_train_1 = np.array([x for x, y in zip(X_train, Y_train) if y == 1])

mean_0 = np.mean(X_train_0, axis = 0)
mean_1 = np.mean(X_train_1, axis = 0)  

# Compute in-class covariance
data_dim = X_train.shape[1]
cov_0 = np.zeros((data_dim, data_dim))
cov_1 = np.zeros((data_dim, data_dim))

for x in X_train_0:
    cov_0 += np.dot(np.transpose([x - mean_0]), [x - mean_0]) / X_train_0.shape[0]
for x in X_train_1:
    cov_1 += np.dot(np.transpose([x - mean_1]), [x - mean_1]) / X_train_1.shape[0]

# Shared covariance is taken as a weighted average of individual in-class covariance.
cov = (cov_0 * X_train_0.shape[0] + cov_1 * X_train_1.shape[0]) / (X_train_0.shape[0] + X_train_1.shape[0])

**Computing weights and bias**

In [28]:
# Compute inverse of covariance matrix.
# Since covariance matrix may be nearly singular, np.linalg.inv() may give a large numerical error.
# Via SVD decomposition, one can get matrix inverse efficiently and accurately.
u, s, v = np.linalg.svd(cov, full_matrices=False)
inv = np.dot(v.T * 1 / s, u.T)

# Directly compute weights and bias
w = np.dot(inv, mean_0 - mean_1)
b =  (-0.5) * np.dot(mean_0, np.dot(inv, mean_0)) + 0.5 * np.dot(mean_1, np.dot(inv, mean_1))\
    + np.log(float(X_train_0.shape[0]) / X_train_1.shape[0]) 

# Compute accuracy on training set
Y_train_pred = 1 - _predict(X_train, w, b)
print('Training accuracy: {}'.format(_accuracy(Y_train_pred, Y_train)))

Training accuracy: 0.8719219994102034


In [0]:
np.save('weights.npy', w)
np.save('cons.npy',b)

**Prediction**

In [0]:
# Predict testing labels
weights = np.load('weights.npy')
cons = np.load('cons.npy')
predictions = 1 - _predict(X_test, w, b)

In [0]:
df = pd.DataFrame(np.empty((predictions.shape[0], 2)), index = np.arange(predictions.shape[0]) + 1, columns = ['id', 'label'], dtype = int)
for i in range (predictions.shape[0]):
  df.iloc[i, 0] = i
  df.iloc[i, 1] = predictions[i]
df.to_csv('submission.csv', index = False)